In [1]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath('../scripts'))
from preprocessing import *

In [2]:
# data pre processing
train = pd.read_csv('../data/train.csv', low_memory=False)
preprocessing_scaler(train)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,IsWeekend,IsHoliday,DaysToHoliday,DaysAfterHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1,False,0,NaN,5.0
1,2,5,2015-07-31,6064,625,1,1,0,1,False,0,NaN,5.0
2,3,5,2015-07-31,8314,821,1,1,0,1,False,0,NaN,5.0
3,4,5,2015-07-31,13995,1498,1,1,0,1,False,0,NaN,5.0
4,5,5,2015-07-31,4822,559,1,1,0,1,False,0,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1,False,1,4.0,NaN
1017205,1112,2,2013-01-01,0,0,0,0,a,1,False,1,4.0,NaN
1017206,1113,2,2013-01-01,0,0,0,0,a,1,False,1,4.0,NaN
1017207,1114,2,2013-01-01,0,0,0,0,a,1,False,1,4.0,NaN


In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline

In [16]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Load the datasets
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
store = pd.read_csv('../data/store.csv')

# Merge the store dataset with train and test based on 'Store' column
train = pd.merge(train, store, how='left', on='Store')
test = pd.merge(test, store, how='left', on='Store')

# Fill missing values in 'CompetitionDistance' and other columns
train['CompetitionDistance'].fillna(0, inplace=True)
test['CompetitionDistance'].fillna(0, inplace=True)
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

# Handle PromoInterval by creating separate columns for each month
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for month in months:
    train[month] = train['PromoInterval'].apply(lambda x: 1 if month in str(x) else 0)
    test[month] = test['PromoInterval'].apply(lambda x: 1 if month in str(x) else 0)

# Drop the 'PromoInterval' column as it's now encoded
train.drop('PromoInterval', axis=1, inplace=True)
test.drop('PromoInterval', axis=1, inplace=True)

# Convert categorical columns to numeric using LabelEncoder
label_encoder = LabelEncoder()

# Encode categorical columns
categorical_columns = ['StoreType', 'Assortment', 'StateHoliday']
for col in categorical_columns:
    train[col] = label_encoder.fit_transform(train[col].astype(str))
    test[col] = label_encoder.transform(test[col].astype(str))

# Prepare the training data
# Check if 'Customers' exists in the training set
if 'Customers' in train.columns:
    X_train = train.drop(['Sales', 'Date', 'Customers'], axis=1)  # Drop 'Sales', 'Date', and 'Customers'
else:
    X_train = train.drop(['Sales', 'Date'], axis=1)  # Drop 'Sales' and 'Date' only

y_train = train['Sales']

# Prepare the test data
X_test = test.drop(['Id', 'Date'], axis=1)

# Ensure that both X_train and X_test have the same columns
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)  # Fill missing columns with zeros

# Create and fit the pipeline
model_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Scale the data
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))  # Random Forest Regressor
])

# Fit the model on the training data
model_pipeline.fit(X_train, y_train)

# Make predictions on the test data
test_predictions = model_pipeline.predict(X_test)

# Save the predictions to a file
test['PredictedSales'] = test_predictions
test[['Store', 'Date', 'PredictedSales']].to_csv('test_predictions.csv', index=False)

print("Predictions saved to 'test_predictions.csv'")


C:\Users\HP\AppData\Local\Temp\ipykernel_30360\3671796347.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('../data/train.csv')


Predictions saved to 'test_predictions.csv'


In [17]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test, test_predictions)
mse = mean_squared_error(y_test, test_predictions)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")

# Visualize predictions vs actual
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(y_test, test_predictions, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)  # line of equality
plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales')
plt.title('Actual vs. Predicted Sales')
plt.show()

NameError: name 'y_test' is not defined